<a href="https://colab.research.google.com/github/eeolga/deep/blob/main/Lesson5_Time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Time series

In [2]:
import numpy as np
import torch

In [3]:
bikes_numpy = np.loadtxt("/content/hour[1].csv", dtype=np.float32,delimiter= ",",skiprows=1, converters= {1: lambda x: float(x[8:10])})
bikes=torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

N is a parallel secuences of size C (column for 1D data)

N - time axis (one entry per hour)

N - number of samples of sequences of length L    

tensor NxCxL

C=17 (table colomns), L=24 (24 hours a day)

In [4]:
bikes.shape, bikes.stride()

(torch.Size([17379, 17]), (17, 1))

Reshape into 3 axes: day, hour  - 17 columns

In [5]:
daily_bikes = bikes.view (-1, 1, bikes.shape[1]) #replaced 24 to 1 :(
daily_bikes.shape, daily_bikes.stride

(torch.Size([17379, 1, 17]), <function Tensor.stride>)

In [6]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([17379, 17, 1]), (17, 1, 17))

Preparing data for training

In [7]:
first_day=bikes[:24] .long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day [ : , 9 ]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [8]:
weather_onehot.scatter_(dim= 1, index=first_day [:, 9] . unsqueeze ( 1) . long () - 1, value= 1.0)


tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [9]:
torch.cat((bikes[:24], weather_onehot), 1) [:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [10]:
daily_weather_onehot =torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])

In [11]:
daily_weather_onehot.shape

torch.Size([17379, 4, 1])

In [17]:
daily_weather_onehot.scatter_(1.0, daily_bikes[:, 9,:].long() .unsqueeze(1) 2, 1.0)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-17-bcfbb8c6a9f0>, line 1)

In [16]:
daily_weather_onehot.shape

torch.Size([17379, 4, 1])

In [18]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
daily_bikes [: , 9,:] = (daily_bikes[: , 9,:] - 1.0)/ 3.0

Alternative, we can subtract the mean and divide by str dev

In [19]:
temp = daily_bikes[:,10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes [:, 10, : ]= ( (daily_bikes [:, 10, : ]- temp_min) / (temp_max - temp_min))

In [20]:
temp = daily_bikes [:,10, : ]
daily_bikes [:, 10,: ]= ( (daily_bikes [:,10, : ]- torch.mean(temp))/torch.std(temp))